In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import json

In [2]:
spark = SparkSession.builder \
.master("local") \
.config("spark.sql.autoBroadcastJoinThreshold", -1) \
.config("spark.executor.memory", "500mb") \
.appName("Games") \
.getOrCreate()

22/04/28 22:17:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/28 22:17:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df = pd.read_json('./Data/data.json')
df.set_index('id', inplace=True)
df.head(5)

,aggregated_rating,aggregated_rating_count,category,cover,first_release_date,game_modes,genres,name,platforms,rating,rating_count,summary,total_rating,total_rating_count
id,,,,,,,,,,,,,,
1,90.000000,1.0,0,90967,9.535968e+08,[1],"[5, 13, 31]",Thief II: The Metal Age,[6],88.291683,109.0,The ultimate thief is back! Tread softly as yo...,89.145842,110.0
2,70.000000,1.0,0,96746,9.123840e+08,[1],"[13, 31]",Thief: The Dark Project,[6],86.300971,132.0,Thief is a first-person stealth game that like...,78.150485,133.0
3,83.000000,3.0,0,129190,1.085443e+09,[1],"[5, 13, 31]",Thief: Deadly Shadows,"[6, 11]",82.529350,103.0,"In the third instalment of the Thief series, m...",82.764675,106.0
4,63.642857,17.0,0,96744,1.393286e+09,[1],"[5, 31]",Thief,"[6, 9, 12, 48, 49]",69.783356,234.0,There is a rising tide of fear in The City. Ha...,66.713106,251.0
5,NaN,NaN,0,93899,9.141984e+08,"[1, 3]",[12],Baldur's Gate,"[3, 6, 14, 39]",86.813896,256.0,Baldur's Gate is a fantasy role-playing video ...,86.813896,256.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 1 to 1015
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   aggregated_rating        652 non-null    float64
 1   aggregated_rating_count  652 non-null    float64
 2   category                 1000 non-null   int64  
 3   cover                    1000 non-null   int64  
 4   first_release_date       998 non-null    float64
 5   game_modes               1000 non-null   object 
 6   genres                   999 non-null    object 
 7   name                     1000 non-null   object 
 8   platforms                998 non-null    object 
 9   rating                   832 non-null    float64
 10  rating_count             832 non-null    float64
 11  summary                  973 non-null    object 
 12  total_rating             876 non-null    float64
 13  total_rating_count       876 non-null    float64
dtypes: float64(7), int64(2),

In [5]:
df['release_date'] = pd.to_datetime(df['first_release_date'], unit='s')
df['Year'] = df['release_date'].dt.year
df = df.drop(columns=['first_release_date','total_rating','total_rating_count'])

In [6]:
df

,aggregated_rating,aggregated_rating_count,category,cover,game_modes,genres,name,platforms,rating,rating_count,summary,release_date,Year
id,,,,,,,,,,,,,
1,90.000000,1.0,0,90967,[1],"[5, 13, 31]",Thief II: The Metal Age,[6],88.291683,109.0,The ultimate thief is back! Tread softly as yo...,2000-03-21,2000.0
2,70.000000,1.0,0,96746,[1],"[13, 31]",Thief: The Dark Project,[6],86.300971,132.0,Thief is a first-person stealth game that like...,1998-11-30,1998.0
3,83.000000,3.0,0,129190,[1],"[5, 13, 31]",Thief: Deadly Shadows,"[6, 11]",82.529350,103.0,"In the third instalment of the Thief series, m...",2004-05-25,2004.0
4,63.642857,17.0,0,96744,[1],"[5, 31]",Thief,"[6, 9, 12, 48, 49]",69.783356,234.0,There is a rising tide of fear in The City. Ha...,2014-02-25,2014.0
5,NaN,NaN,0,93899,"[1, 3]",[12],Baldur's Gate,"[3, 6, 14, 39]",86.813896,256.0,Baldur's Gate is a fantasy role-playing video ...,1998-12-21,1998.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1011,80.705882,18.0,0,94379,"[1, 2, 3, 4]","[5, 12]",Borderlands 2,"[3, 6, 9, 12, 14, 34, 46, 48]",84.753323,1337.0,A new era of shoot and loot is about to begin....,2012-09-18,2012.0
1012,73.142857,8.0,0,118574,[2],[5],Gotham City Impostors,"[6, 9, 12]",69.765300,27.0,Take to the mean streets of an unhinged Gotham...,2012-01-10,2012.0
1013,79.500000,11.0,0,84601,"[1, 2]","[4, 13, 15, 33]",SoulCalibur V,"[9, 12]",68.678521,65.0,"SOULCALIBUR V, the latest entry in the premier...",2012-01-31,2012.0


In [7]:
cover_df = pd.read_json('./Data/data_url.json')
cover_df.set_index('id', inplace=True)
cover_df.head(5)

,url
id,
3975,//images.igdb.com/igdb/image/upload/t_thumb/z7...
3978,//images.igdb.com/igdb/image/upload/t_thumb/cg...
3979,//images.igdb.com/igdb/image/upload/t_thumb/na...
3980,//images.igdb.com/igdb/image/upload/t_thumb/ei...
3983,//images.igdb.com/igdb/image/upload/t_thumb/t0...


In [8]:
platform_df = pd.read_json('./Data/data_platforms.json')
platform_df.set_index('id', inplace=True)
platform_df.head(5)

,name
id,
3,Linux
4,Nintendo 64
5,Wii
6,PC (Microsoft Windows)
7,PlayStation


In [9]:
genre_df = pd.read_json('./Data/data_genre.json')
genre_df.set_index('id', inplace=True)
genre_df.head(5)

,name
id,
2,Point-and-click
4,Fighting
5,Shooter
7,Music
8,Platform


In [10]:
gamemode_df = pd.read_json('./Data/data_gamemode.json')
gamemode_df.set_index('id', inplace=True)
gamemode_df.head(5)

,name
id,
1,Single player
2,Multiplayer
3,Co-operative
4,Split screen
5,Massively Multiplayer Online (MMO)


In [11]:
df.isnull().sum()

aggregated_rating          348
aggregated_rating_count    348
category                     0
cover                        0
game_modes                   0
genres                       1
name                         0
platforms                    2
rating                     168
rating_count               168
summary                     27
release_date                 2
Year                         2
dtype: int64

In [12]:
cover_df.isnull().sum()

url    0
dtype: int64

In [13]:
genre_df.isnull().sum()

name    0
dtype: int64

In [14]:
gamemode_df.isnull().sum()

name    0
dtype: int64

In [15]:
platform_df.isnull().sum()

name    0
dtype: int64

In [16]:
df = df.dropna()

In [17]:
df.isnull().sum()

aggregated_rating          0
aggregated_rating_count    0
category                   0
cover                      0
game_modes                 0
genres                     0
name                       0
platforms                  0
rating                     0
rating_count               0
summary                    0
release_date               0
Year                       0
dtype: int64